Imports

In [43]:
import os
import numpy as np
import mne
import matplotlib
import matplotlib.pyplot as plt

Ensure Matplotlib uses the Qt5Agg backend for plotting functions

In [44]:
matplotlib.use('Qt5Agg')
annot_kwargs = dict(fontsize=12, fontweight='bold',
                    xycoords="axes fraction", ha='right', va='center')
fontsize = 8
params = {'font.size': fontsize,
          'axes.labelsize': fontsize,
          'legend.fontsize': fontsize,
          'xtick.labelsize': fontsize,
          'ytick.labelsize': fontsize,
          'axes.titlesize': fontsize + 2,
          'figure.max_open_warning': 200,
          'axes.spines.top': False,
          'axes.spines.right': False,
          'axes.grid': True,
          'lines.linewidth': 1
          }
plt.rcParams.update(params)

Configure to use Cuda

In [45]:
try:
    mne.set_config('MNE_USE_CUDA', 'true')
except TypeError as err:
    print(err)

Choose channels from the raw edf file to use

In [46]:
# good channels: [4,5,6,7,8,9,10,11,12,13,14,15,16,17]
# all channels ['TIME_STAMP_s', 'TIME_STAMP_ms', 'COUNTER', 'INTERPOLATED', 'AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4', 'HighBitFlex', 'SaturationFlag', 'RAW_CQ', 'BATTERY', 'BATTERY_PERCENT', 'MarkerIndex', 'MarkerType', 'MarkerValueInt', 'MARKER_HARDWARE', 'CQ_AF3', 'CQ_F7', 'CQ_F3', 'CQ_FC5', 'CQ_T7', 'CQ_P7', 'CQ_O1', 'CQ_O2', 'CQ_P8', 'CQ_T8', 'CQ_FC6', 'CQ_F4', 'CQ_F8', 'CQ_AF4', 'CQ_OVERALL', 'EQ_SampleRateQua', 'EQ_OVERALL', 'EQ_AF3', 'EQ_F7', 'EQ_F3', 'EQ_FC5', 'EQ_T7', 'EQ_P7', 'EQ_O1', 'EQ_O2', 'EQ_P8', 'EQ_T8', 'EQ_FC6', 'EQ_F4', 'EQ_F8', 'EQ_AF4', 'CQ_CMS', 'CQ_DRL']
stim_channels = ['MarkerIndex', 'MarkerType', 'MarkerValueInt', 'MARKER_HARDWARE']
exclude = ['TIME_STAMP_s', 'TIME_STAMP_ms', 'COUNTER', 'INTERPOLATED', 'HighBitFlex', 'SaturationFlag', 'RAW_CQ', 'BATTERY', 'BATTERY_PERCENT', 'CQ_AF3', 'CQ_F7', 'CQ_F3', 'CQ_FC5', 'CQ_T7', 'CQ_P7', 'CQ_O1', 'CQ_O2', 'CQ_P8', 'CQ_T8', 'CQ_FC6', 'CQ_F4', 'CQ_F8', 'CQ_AF4', 'CQ_OVERALL', 'EQ_SampleRateQua', 'EQ_OVERALL', 'EQ_AF3', 'EQ_F7', 'EQ_F3', 'EQ_FC5', 'EQ_T7', 'EQ_P7', 'EQ_O1', 'EQ_O2', 'EQ_P8', 'EQ_T8', 'EQ_FC6', 'EQ_F4', 'EQ_F8', 'EQ_AF4', 'CQ_CMS', 'CQ_DRL']
#  eog_channels = ['AF3', 'AF4']

Import EDF

In [47]:
#  edf file name here
edf_file_name = 'EEG-Game_Josh Schrock_EPOCFLEX-F0000172_EPOCFLEX_123045_2022.06.21T14.54.07.04.00'
data_raw_file = f"{os.getcwd()}\\BCI Evasion 2\\EEGExports\\{edf_file_name}.edf"
raw = mne.io.read_raw_edf(data_raw_file, stim_channel=stim_channels, exclude=exclude, preload=True)

Extracting EDF parameters from C:\Users\larx\PycharmProjects\EEG-Game\BCI Evasion 2\EEGExports\EEG-Game_Josh Schrock_EPOCFLEX-F0000172_EPOCFLEX_123045_2022.06.21T13.09.47.04.00.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 4351  =      0.000 ...    33.992 secs...


Set up the montage for this data

In [48]:
# Form the 10-20 montage
mont1020 = mne.channels.make_standard_montage('standard_1020')
# Choose what channels you want to keep
# Make sure that these channels exist e.g. T1 does not exist in the standard 10-20 EEG system!
kept_channels = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
ind = [i for (i, channel) in enumerate(mont1020.ch_names) if channel in kept_channels]
mont1020_new = mont1020.copy()
# Keep only the desired channels
mont1020_new.ch_names = [mont1020.ch_names[x] for x in ind]
kept_channel_info = [mont1020.dig[x+3] for x in ind]
# Keep the first three rows as they are the fiducial points information
mont1020_new.dig = mont1020.dig[0:3]+kept_channel_info

Apply montage to the raw data and plot

In [49]:
raw = raw.set_montage(mont1020_new)
raw.plot_sensors(kind='3d', ch_type='eeg')
raw.plot()
raw.plot_psd()

Opening raw-browser...
Effective window size : 16.000 (s)
Closing raw-browser...
Channels marked as bad:
none


<MNELineFigure size 1000x350 with 2 Axes>

Preprocess Data

In [50]:
filtered_raw = raw.copy().filter(
    1, 40, l_trans_bandwidth='auto', h_trans_bandwidth='auto',
    filter_length='auto', phase='zero', fir_window='hamming',
    fir_design='firwin')
fid, ax = plt.subplots(2)
raw.plot_psd(ax=ax[0], show=False)
filtered_raw.plot_psd(ax=ax[1], show=False)
ax[0].set_title('Raw')
ax[1].set_title('Filtered Raw')

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 423 samples (3.305 sec)

Effective window size : 16.000 (s)
Effective window size : 16.000 (s)


Text(0.5, 1.0, 'Filtered Raw')

Extract events from the raw data

In [51]:
events = mne.find_events(filtered_raw)
print(events)

17 events found
Event IDs: [     1      2      3      4      5      6      7      8      9     10
     11 131062 131064 131066 131068 131069 131071]
17 events found
Event IDs: [1 2]
5 events found
Event IDs: [1]
[[   254      0      2]
 [   254      0      1]
 [   449      0      2]
 [   449      0      1]
 [   704      0 131071]
 [   704      0      2]
 [   864      0      3]
 [   864      0      2]
 [  1025      0      2]
 [  1025      0 131069]
 [  1159      0      2]
 [  1159      0      4]
 [  1241      0      2]
 [  1241      0 131068]
 [  1260      0      5]
 [  1260      0      1]
 [  1598      0      6]
 [  1598      0      2]
 [  1643      0      2]
 [  1643      0 131066]
 [  1922      0      1]
 [  1922      0      7]
 [  2032      0      8]
 [  2032      0      2]
 [  2114      0      2]
 [  2114      0 131064]
 [  3260      0      9]
 [  3260      0      1]
 [  3423      0     10]
 [  3423      0      2]
 [  3520      0 131062]
 [  3520      0      2]
 [  4126      0     

C:\Users\larx\AppData\Local\Temp\ipykernel_4052\2919268591.py:1: RuntimeWarning: Some events are duplicated in your different stim channels. 5 events were ignored during deduplication.
  events = mne.find_events(filtered_raw)


In [52]:
raw.info

<Info | 8 non-empty values
 bads: []
 ch_names: AF3, F7, F3, FC5, T7, P7, O1, O2, P8, T8, FC6, F4, F8, AF4, ...
 chs: 14 EEG, 4 Stimulus
 custom_ref_applied: False
 dig: 17 items (3 Cardinal, 14 EEG)
 highpass: 0.0 Hz
 lowpass: 64.0 Hz
 meas_date: 2022-06-21 13:09:47 UTC
 nchan: 18
 projs: []
 sfreq: 128.0 Hz
>